# Character-Level LSTM in PyTorch

It will be train on **Notre-Dame de Paris** from Victor Hugo. The purpose pof this model is to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [2]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F


## Load in Data


### Tokenization

Convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [3]:
# open text file and read in data as `text`
with open('data/hugo.txt', 'r') as f:
    text = f.read()

Now we have the text, encode it as integers.

In [4]:
# encode the text and map each character to an integer and vice versa

# we create two dictonaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])

In [5]:
text[:100]

'Three hundred and forty-eight years, six months, and nineteen days ago\nto-day, the Parisians awoke t'

And we can see those same characters encoded as integers.

In [6]:
encoded[:100]

array([42,  5, 90, 39, 39, 87,  5, 74, 73, 66, 90, 39, 66, 87, 67, 73, 66,
       87, 40, 57, 90, 31, 86,  1, 39, 71, 21,  5, 31, 87, 86, 39, 67, 90,
       28, 38, 87, 28, 71, 45, 87, 60, 57, 73, 31,  5, 28, 38, 87, 67, 73,
       66, 87, 73, 71, 73, 39, 31, 39, 39, 73, 87, 66, 67, 86, 28, 87, 67,
       21, 57, 44, 31, 57,  1, 66, 67, 86, 38, 87, 31,  5, 39, 87, 24, 67,
       90, 71, 28, 71, 67, 73, 28, 87, 67, 83, 57,  7, 39, 87, 31])

## Pre-processing the data

The LSTM expects an input that is **one-hot encoded** meaning that each character is converted into an integer (via our created dictionary) and then converted into a column vector where only it's corresponsing integer index will have the value of 1 and the rest of the vector will be filled with 0's. 


In [21]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `n_seqs` (also refered to as "batch size" in other places). Each of those sequences will be `n_steps` long.

### Creating Batches

**1. The first thing we need to do is discard some of the text so we only have completely full batches. **

Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the total number of batches, $K$, we can make from the array `arr`, you divide the length of `arr` by the number of characters per batch. Once you know the number of batches, you can get the total number of characters to keep from `arr`, $N * M * K$.

**2. After that, we need to split `arr` into $N$ sequences. ** 

You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences, so let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$.

**3. Now that we have this array, we can iterate through it to get our batches. **

The idea is each batch is a $N \times M$ window on the $N \times (M * K)$ array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.


In [8]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Your Implementation

Now I'll make some data sets and we can check out what's going on as we batch data. Here, as an example, I'm going to use a batch size of 10 and 50 sequence steps.

In [22]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [10]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[42  5 90 39 39 87  5 74 73 66]
 [71 73 87 31  5 39 87 28  5 67]
 [39 54 67 74 28 39 87 57 40 87]
 [87  5 71 28 87 60 67 28 31 39]
 [89 39 28 87 10 40 57 90 87 31]
 [54 39 38 87 57 73 87 28 39 89]
 [40 40 71 54 39 90 44 46 67 71]
 [71 90 39 87 57 40 44 36 67 71]
 [71 28 44 31 57 83 39 90 87 57]
 [39 71 21  5 82 57 90 71 73 21]]

y
 [[ 5 90 39 39 87  5 74 73 66 90]
 [73 87 31  5 39 87 28  5 67 66]
 [54 67 74 28 39 87 57 40 87 31]
 [ 5 71 28 87 60 67 28 31 39 90]
 [39 28 87 10 40 57 90 87 31  5]
 [39 38 87 57 73 87 28 39 89 89]
 [40 71 54 39 90 44 46 67 71 66]
 [90 39 87 57 40 44 36 67 71 73]
 [28 44 31 57 83 39 90 87 57 73]
 [71 21  5 82 57 90 71 73 21 87]]


---
## Defining the network with PyTorch

Below is where you'll define the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>

Next, you'll use PyTorch to define the architecture of the network. We start by defining the layers and operations we want. Then, define a method for the forward pass. You've also been given a method for predicting characters.

### Model Structure

In `__init__` the suggested structure is as follows:
* Create and store the necessary dictionaries (this has been done for you)
* Define an LSTM layer that takes as params: an input size (the number of characters), a hidden layer size `n_hidden`, a number of layers `n_layers`, a dropout probability `drop_prob`, and a batch_first boolean (True, since we are batching)
* Define a dropout layer with `dropout_prob`
* Define a fully-connected layer with params: input size `n_hidden` and output size (the number of characters)
* Finally, initialize the weights (again, this has been given)

Note that some parameters have been named and given in the `__init__` function, and we use them and store them by doing something like `self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

You can create a basic LSTM cell as follows

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

where `input_size` is the number of characters this cell expects to see as sequential input, and `n_hidden` is the number of units in the hidden layers in the cell. And we can add dropout by adding a dropout parameter with a specified probability; this will automatically add dropout to the inputs or outputs. Finally, in the `forward` function, we can stack up the LSTM cells into layers using `.view`. With this, you pass in a list of cells and it will send the output of one cell into the next cell.

We also need to create an initial cell state of all zeros. This is done like so

```python
self.init_weights()
```

In [11]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # initialize the weights
        self.init_weights()
      
    
    def forward(self, x, hc):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hc`. '''
        
        ## Get x, and the new hidden state (h, c) from the lstm
        x, (h, c) = self.lstm(x, hc)
        
        ## pass x through a droupout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs using view
        x = x.view(x.size()[0]*x.size()[1], self.n_hidden)
        
        ## put x through the fully-connected layer
        x = self.fc(x)
        
        # return x and the hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
        
            Returns the predicted character and the hidden state.
        '''
        if cuda:
            self.cuda()
        else:
            self.cpu()
        
        if h is None:
            h = self.init_hidden(1)
        
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        inputs = torch.from_numpy(x)
        if cuda:
            inputs = inputs.cuda()
        
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)

        p = F.softmax(out, dim=1).data
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fully connected layer '''
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())
        

### A note on the `predict`  function

The output of our RNN is from a fully-connected layer and it outputs a **distribution of next-character scores**.

To actually get the next character, we apply a softmax function, which gives us a *probability* distribution that we can then sample to predict the next character.

In [12]:
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        n_seqs: Number of mini-sequences per mini-batch, aka batch size
        n_steps: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        cuda: Train with CUDA on a GPU
        print_every: Number of steps for printing training and validation loss
    
    '''
    
    net.train()
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if cuda:
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        for x, y in get_batches(data, n_seqs, n_steps):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net.forward(inputs, h)
            loss = criterion(output, targets.view(n_seqs*n_steps))

            loss.backward()
            
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
            if counter % print_every == 0:
                
                # Get validation loss
                val_h = net.init_hidden(n_seqs)
                val_losses = []
                for x, y in get_batches(val_data, n_seqs, n_steps):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if cuda:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net.forward(inputs, val_h)
                    val_loss = criterion(output, targets.view(n_seqs*n_steps))
                
                    val_losses.append(val_loss.item())
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Time to train

Now we can actually train the network. First we'll create the network itself, with some given hyperparameters. Then, define the mini-batches sizes (number of sequences and number of steps), and start the training. With the train function, we can set the number of epochs, the learning rate, and other parameters. Also, we can run the training on a GPU by setting `cuda=True`.

In [13]:
if 'net' in locals():
    del net

In [14]:
# define and print the net
net = CharRNN(chars, n_hidden=512, n_layers=2)
print(net)

CharRNN(
  (lstm): LSTM(93, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=93, bias=True)
)


In [15]:
n_seqs, n_steps = 32, 100

train(net, encoded, epochs=25, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=True, print_every=10)

Epoch: 1/25... Step: 10... Loss: 3.3832... Val Loss: 3.3320
Epoch: 1/25... Step: 20... Loss: 3.2724... Val Loss: 3.2347
Epoch: 1/25... Step: 30... Loss: 3.1528... Val Loss: 3.1208
Epoch: 1/25... Step: 40... Loss: 3.0708... Val Loss: 2.9943
Epoch: 1/25... Step: 50... Loss: 2.8911... Val Loss: 2.8460
Epoch: 1/25... Step: 60... Loss: 2.7531... Val Loss: 2.7217
Epoch: 1/25... Step: 70... Loss: 2.6946... Val Loss: 2.6420
Epoch: 1/25... Step: 80... Loss: 2.6821... Val Loss: 2.5851
Epoch: 1/25... Step: 90... Loss: 2.5527... Val Loss: 2.5363
Epoch: 1/25... Step: 100... Loss: 2.5387... Val Loss: 2.5033
Epoch: 1/25... Step: 110... Loss: 2.4306... Val Loss: 2.4678
Epoch: 1/25... Step: 120... Loss: 2.4326... Val Loss: 2.4437
Epoch: 1/25... Step: 130... Loss: 2.4004... Val Loss: 2.4115
Epoch: 1/25... Step: 140... Loss: 2.3949... Val Loss: 2.3892
Epoch: 1/25... Step: 150... Loss: 2.3521... Val Loss: 2.3714
Epoch: 1/25... Step: 160... Loss: 2.3262... Val Loss: 2.3477
Epoch: 1/25... Step: 170... Loss:

Epoch: 5/25... Step: 1350... Loss: 1.6697... Val Loss: 1.6457
Epoch: 5/25... Step: 1360... Loss: 1.6740... Val Loss: 1.6383
Epoch: 5/25... Step: 1370... Loss: 1.6427... Val Loss: 1.6346
Epoch: 5/25... Step: 1380... Loss: 1.6943... Val Loss: 1.6350
Epoch: 5/25... Step: 1390... Loss: 1.6532... Val Loss: 1.6332
Epoch: 5/25... Step: 1400... Loss: 1.7033... Val Loss: 1.6256
Epoch: 5/25... Step: 1410... Loss: 1.6531... Val Loss: 1.6302
Epoch: 5/25... Step: 1420... Loss: 1.7076... Val Loss: 1.6268
Epoch: 5/25... Step: 1430... Loss: 1.6323... Val Loss: 1.6222
Epoch: 5/25... Step: 1440... Loss: 1.6662... Val Loss: 1.6216
Epoch: 5/25... Step: 1450... Loss: 1.6373... Val Loss: 1.6216
Epoch: 5/25... Step: 1460... Loss: 1.6235... Val Loss: 1.6141
Epoch: 5/25... Step: 1470... Loss: 1.6596... Val Loss: 1.6115
Epoch: 5/25... Step: 1480... Loss: 1.5907... Val Loss: 1.6115
Epoch: 6/25... Step: 1490... Loss: 1.6182... Val Loss: 1.6064
Epoch: 6/25... Step: 1500... Loss: 1.6190... Val Loss: 1.6063
Epoch: 6

Epoch: 10/25... Step: 2680... Loss: 1.4196... Val Loss: 1.4633
Epoch: 10/25... Step: 2690... Loss: 1.4358... Val Loss: 1.4593
Epoch: 10/25... Step: 2700... Loss: 1.4945... Val Loss: 1.4600
Epoch: 10/25... Step: 2710... Loss: 1.5206... Val Loss: 1.4628
Epoch: 10/25... Step: 2720... Loss: 1.4767... Val Loss: 1.4649
Epoch: 10/25... Step: 2730... Loss: 1.4400... Val Loss: 1.4590
Epoch: 10/25... Step: 2740... Loss: 1.4293... Val Loss: 1.4589
Epoch: 10/25... Step: 2750... Loss: 1.4508... Val Loss: 1.4555
Epoch: 10/25... Step: 2760... Loss: 1.4074... Val Loss: 1.4535
Epoch: 10/25... Step: 2770... Loss: 1.4702... Val Loss: 1.4563
Epoch: 10/25... Step: 2780... Loss: 1.4369... Val Loss: 1.4549
Epoch: 10/25... Step: 2790... Loss: 1.4184... Val Loss: 1.4555
Epoch: 10/25... Step: 2800... Loss: 1.4948... Val Loss: 1.4525
Epoch: 10/25... Step: 2810... Loss: 1.4897... Val Loss: 1.4501
Epoch: 10/25... Step: 2820... Loss: 1.3982... Val Loss: 1.4514
Epoch: 10/25... Step: 2830... Loss: 1.4660... Val Loss:

Epoch: 14/25... Step: 3990... Loss: 1.3567... Val Loss: 1.3971
Epoch: 14/25... Step: 4000... Loss: 1.4081... Val Loss: 1.3993
Epoch: 14/25... Step: 4010... Loss: 1.3212... Val Loss: 1.4020
Epoch: 14/25... Step: 4020... Loss: 1.3398... Val Loss: 1.3986
Epoch: 14/25... Step: 4030... Loss: 1.3333... Val Loss: 1.3962
Epoch: 14/25... Step: 4040... Loss: 1.3212... Val Loss: 1.4008
Epoch: 14/25... Step: 4050... Loss: 1.3052... Val Loss: 1.4028
Epoch: 14/25... Step: 4060... Loss: 1.3876... Val Loss: 1.3956
Epoch: 14/25... Step: 4070... Loss: 1.3029... Val Loss: 1.3875
Epoch: 14/25... Step: 4080... Loss: 1.3001... Val Loss: 1.3908
Epoch: 14/25... Step: 4090... Loss: 1.3585... Val Loss: 1.3960
Epoch: 14/25... Step: 4100... Loss: 1.3367... Val Loss: 1.3970
Epoch: 14/25... Step: 4110... Loss: 1.3132... Val Loss: 1.3901
Epoch: 14/25... Step: 4120... Loss: 1.4024... Val Loss: 1.3930
Epoch: 14/25... Step: 4130... Loss: 1.3310... Val Loss: 1.3935
Epoch: 14/25... Step: 4140... Loss: 1.2829... Val Loss:

Epoch: 18/25... Step: 5300... Loss: 1.2525... Val Loss: 1.3623
Epoch: 18/25... Step: 5310... Loss: 1.2798... Val Loss: 1.3573
Epoch: 18/25... Step: 5320... Loss: 1.3235... Val Loss: 1.3735
Epoch: 18/25... Step: 5330... Loss: 1.2451... Val Loss: 1.3660
Epoch: 18/25... Step: 5340... Loss: 1.3293... Val Loss: 1.3642
Epoch: 19/25... Step: 5350... Loss: 1.2803... Val Loss: 1.3635
Epoch: 19/25... Step: 5360... Loss: 1.2442... Val Loss: 1.3666
Epoch: 19/25... Step: 5370... Loss: 1.2333... Val Loss: 1.3728
Epoch: 19/25... Step: 5380... Loss: 1.2972... Val Loss: 1.3657
Epoch: 19/25... Step: 5390... Loss: 1.3572... Val Loss: 1.3638
Epoch: 19/25... Step: 5400... Loss: 1.2963... Val Loss: 1.3657
Epoch: 19/25... Step: 5410... Loss: 1.2720... Val Loss: 1.3668
Epoch: 19/25... Step: 5420... Loss: 1.3037... Val Loss: 1.3705
Epoch: 19/25... Step: 5430... Loss: 1.2823... Val Loss: 1.3635
Epoch: 19/25... Step: 5440... Loss: 1.2617... Val Loss: 1.3642
Epoch: 19/25... Step: 5450... Loss: 1.2593... Val Loss:

Epoch: 23/25... Step: 6610... Loss: 1.2676... Val Loss: 1.3560
Epoch: 23/25... Step: 6620... Loss: 1.2242... Val Loss: 1.3612
Epoch: 23/25... Step: 6630... Loss: 1.2065... Val Loss: 1.3571
Epoch: 23/25... Step: 6640... Loss: 1.1439... Val Loss: 1.3588
Epoch: 23/25... Step: 6650... Loss: 1.1628... Val Loss: 1.3627
Epoch: 23/25... Step: 6660... Loss: 1.2597... Val Loss: 1.3655
Epoch: 23/25... Step: 6670... Loss: 1.1960... Val Loss: 1.3581
Epoch: 23/25... Step: 6680... Loss: 1.1546... Val Loss: 1.3651
Epoch: 23/25... Step: 6690... Loss: 1.2122... Val Loss: 1.3686
Epoch: 23/25... Step: 6700... Loss: 1.2026... Val Loss: 1.3531
Epoch: 23/25... Step: 6710... Loss: 1.2041... Val Loss: 1.3604
Epoch: 23/25... Step: 6720... Loss: 1.1903... Val Loss: 1.3580
Epoch: 23/25... Step: 6730... Loss: 1.1785... Val Loss: 1.3634
Epoch: 23/25... Step: 6740... Loss: 1.2696... Val Loss: 1.3621
Epoch: 23/25... Step: 6750... Loss: 1.1730... Val Loss: 1.3551
Epoch: 23/25... Step: 6760... Loss: 1.1989... Val Loss:

## Getting the best model

To set your hyperparameters to get the best performance, you'll want to watch the training and validation losses. If your training loss is much lower than the validation loss, you're overfitting. Increase regularization (more dropout) or use a smaller network. If the training and validation losses are close, you're underfitting so you can increase the size of the network.

## Hyperparameters

Here are the hyperparameters for the network.

In defining the model:
* `n_hidden` - The number of units in the hidden layers.
* `n_layers` - Number of hidden LSTM layers to use.

We assume that dropout probability and learning rate will be kept at the default, in this example.

And in training:
* `n_seqs` - Number of sequences running through the network in one pass.
* `n_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lr` - Learning rate for training

Here's some good advice from Andrej Karpathy on training the network.
> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `n_hidden` and `n_layers`. I would advise that you always use `n_layers` of either 2/3. The `n_hidden` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `n_hidden` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.

After training, we'll save the model so we can load it again later if we need too. Here I'm saving the parameters needed to create the same architecture, the hidden layer hyperparameters and the text characters.

In [24]:
model_name = 'rnn_25_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

## Sampling

Now that the model is trained, we'll want to sample from it. To sample, we pass in a character and have the network predict the next character. Then we take that character, pass it back in, and get another predicted character. Just keep doing this and you'll generate a bunch of text!

### Top K sampling

Our predictions come from a categorcial probability distribution over all the possible characters. We can make the sample text and make it more reasonable to handle (with less variables) by only considering some $K$ most probable characters. This will prevent the network from giving us completely absurd characters while allowing it to introduce some noise and randomness into the sampled text.

Typically you'll want to prime the network so you can build up a hidden state. Otherwise the network will start out generating characters at random. In general the first bunch of characters will be a little rough since it hasn't built up a long history of characters to predict from.

In [25]:
def sample(net, size, prime='The', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [28]:
print(sample(net, 1000, prime='Quasimodo', top_k=5, cuda=True))

Quasimodo,
he was to bonder to the power of those of the counsing of the bonfire.
All at once, she was still their song out and blows that only one has
assured the meature of a time that is that they were peaced
all at the same time, a more staircased at the fiftienth close in the cardinal,
we comple the black mair for a seculiar of magic; there will be already
straight breaks to her his. The archdeacon was even to story; then
he had stunged with tranquil, all that he crimbleds, in the chamber of
all the people of the church which closed the church of Notre-Dame,” asked the
captain, then the second to the contents of his pleasant time
to the child who would approach a state of telling without
clurched with attention, the more three towers. The garden was but his
bald his hands on the part of to the treature of his captain; who had
not taken sure, that the passion was on a letter on all the saints in
the procurator of the popular tower of the pavement of Pharis, which was stept
in the p

## Loading a checkpoint

In [27]:
# Here we have loaded in a model 
with open('rnn_25_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [29]:
print(sample(loaded, 1000, cuda=True, top_k=5, prime="From the"))

From the
basking of a transformity. The painture, one’s extenism of the secund
and a part of south of the threng in his hand with such a point to crouch
the presence of the crowd, though something one of his hideous fees
of the cold and presting halter of the body of the pray, the most
which is crassed and too attitude, by a soul of a pieuce, and satisfated to
his shoulders; the sombre shop which shums and the middle of the poor façade
was the piece of streets, thick of would say what a second.

The gypsy glassed himself in the middle of the gypsy, when the
cracked place. All at once, he wished and to all this in face
of a few miled with a call and walm of stone around his choice,
and his stronged and they were the last middle of the starse. Hurch
to the fire whose hands of the book of the book, the gypsy, with the crocchanal
and door of the priest’s horrs, it would always began to door. It was, and we
have not a cardent and cluther of plain and the perfust of a motion, it
was the prov